In [1]:
import pandas as pd
import os
from fuzzywuzzy import fuzz, process
from pandas_profiling import ProfileReport

In [2]:
PATH = 'c:\\Users\\calvin\\Documents\\GitHub\\springboard\\champions_league_luck'
starturl = 'https://kassiesa.net/uefa/data/method2/trank2000.html'
kassiesaPath = PATH + '/data/raw/kassiesa/'

In [3]:
rankdict = {}
for page in os.listdir(kassiesaPath):
    rankdict[page[5:-5]] = pd.read_html(kassiesaPath + page)[0]

In [4]:
# lets slim these down to country and ranking
for k, v in rankdict.items():
    print(v.head(3))

   Unnamed: 0       Unnamed: 1 Unnamed: 2    95/96   96/97    97/98    98/99  \
0         1.0         Juventus        Ita  22.7500  27.071  23.1425  20.0000   
1         2.0     FC Barcelona        Esp  22.5000  22.800   9.2140  12.0000   
2         3.0  Bayern MÃ¼nchen        Ger  28.4165   6.571  16.5000  22.1785   

     99/00  ranking  
0  17.0000  109.963  
1  37.2855  103.799  
2  29.5355  103.201  
   Unnamed: 0         Unnamed: 1 Unnamed: 2   96/97    97/98    98/99  \
0         1.0        Real Madrid        Esp   5.800  25.2140  16.0000   
1         2.0  Manchester United        Eng  15.700  17.8925  23.7185   
2         3.0    Bayern MÃ¼nchen        Ger   6.571  16.5000  22.1785   

     99/00    00/01  ranking  
0  36.2855  31.3055  114.605  
1  28.2500  25.0830  110.644  
2  29.5355  35.5310  110.316  
   Unnamed: 0         Unnamed: 1 Unnamed: 2    97/98    98/99    99/00  \
0         1.0        Real Madrid        Esp  25.2140  16.0000  36.2855   
1         2.0    Bayern MÃ

In [5]:
# looks like team switches unnamed columns
for k, v in rankdict.items():
    for col in v.columns:
        if v[col].dtype == 'object':
            if 'Real Madrid' in v[col].values:
                print(col) 
                print(k)

Unnamed: 1
2000
Unnamed: 1
2001
Unnamed: 1
2002
Unnamed: 1
2003
Unnamed: 1
2004
Unnamed: 1
2005
Unnamed: 1
2006
Unnamed: 1
2007
Unnamed: 1
2008
Unnamed: 1
2009
Unnamed: 1
2010
Unnamed: 1
2011
Unnamed: 1
2012
Unnamed: 1
2013
Unnamed: 1
2014
Unnamed: 1
2015
Unnamed: 1
2016
Unnamed: 1
2017
Unnamed: 2
2018
Unnamed: 2
2019
Unnamed: 2
2020
Unnamed: 2
2021
Unnamed: 2
2022
Unnamed: 2
2023
Unnamed: 2
2024


In [6]:
# looks like after 2018 they changed the column for the team name
# looking back up at the print out of the dataframes, it also appears that the ranking column changed from ranking to TotalPoints
for k, v in rankdict.items():
    if int(k) < 2018:
        rankdict[k] = v[['Unnamed: 1', 'Unnamed: 2', 'ranking']]
        rankdict[k].rename(columns={'Unnamed: 1': 'team', 'Unnamed: 2': 'country', 'ranking': f'coef{k}'}, inplace=True)
    elif int(k) < 2021:
        rankdict[k] = v[['Unnamed: 2', 'Unnamed: 3', 'TotalPoints']]
        rankdict[k].rename(columns={'Unnamed: 2': 'team', 'Unnamed: 3': 'country', 'TotalPoints': f'coef{k}'}, inplace=True)

C:\Users\calvin\Anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [7]:
# remove past 2020 of the dictionary
_ = [rankdict.pop(k) for k in rankdict.copy() if int(k) > 2020]

In [8]:
# remove countries and country codes
for k, v in rankdict.items():
    rankdict[k] = v[v['country'].notnull()]
    rankdict[k].drop(columns=['country'], inplace=True)

C:\Users\calvin\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
rankdict.keys()

dict_keys(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])

In [10]:
# lets combine them
ranks = pd.DataFrame()
for k, v in rankdict.items():
    if int(k) == 2000:
        ranks = v
    else:
        ranks = ranks.merge(v, on='team', how='outer')


In [11]:
ranks.head()

,team,coef2000,coef2001,coef2002,coef2003,coef2004,coef2005,coef2006,coef2007,coef2008,...,coef2011,coef2012,coef2013,coef2014,coef2015,coef2016,coef2017,coef2018,coef2019,coef2020
0,Juventus,109.963,98.119,91.334,100.155,84.531,93.191,107.020,92.808,66.934,...,47.110,46.996,70.829,80.387,95.102,107.087,140.666,126.0,124.0,117.0
1,FC Barcelona,103.799,108.605,116.233,140.769,134.350,117.326,127.006,119.374,117.837,...,141.465,157.837,157.605,157.542,164.999,159.142,151.999,132.0,138.0,128.0
2,Bayern MÃ¼nchen,103.201,110.316,133.495,124.566,105.331,97.166,80.960,73.640,92.078,...,118.887,133.037,146.922,154.328,154.883,163.035,154.899,135.0,128.0,136.0
3,Real Madrid,99.799,114.605,147.233,151.769,146.350,131.326,120.006,104.374,93.837,...,103.465,121.837,136.605,161.542,171.999,176.142,176.999,162.0,146.0,134.0
4,Lazio,94.963,105.119,106.334,106.155,83.531,69.191,57.020,51.808,38.934,...,23.110,29.996,41.829,52.387,49.102,63.087,56.666,41.0,37.0,41.0


In [12]:
ranks.shape

(747, 22)

In [13]:
# maybe there are differnet names for the same column, lets use fuzzy wuzzy
for i in ranks.team:
    rank = process.extractBests(i, ranks.team, scorer=fuzz.token_set_ratio, score_cutoff=80 )
    if len(rank) > 2:
        print(rank)

[('Arsenal', 100, 30), ('Arsenal Kiev', 100, 256), ('Arsenal Tula', 100, 730)]
[('PAOK Thessaloniki', 100, 96), ('Aris Thessaloniki', 83, 119), ('Iraklis Thessaloniki', 83, 231)]
[('Aris Thessaloniki', 100, 119), ('Iraklis Thessaloniki', 86, 231), ('PAOK Thessaloniki', 83, 96)]
[('MTK Budapest', 100, 123), ('Vasas Budapest', 80, 245), ('HonvÃ©d Budapest', 80, 395)]
[('NK Zagreb', 100, 187), ('Dinamo Zagreb', 80, 88), ('Lokomotiva Zagreb', 80, 583)]
[('KR Reykjavik', 100, 223), ('Fram Reykjavik', 86, 412), ('Fylkir Reykjavik', 86, 499), ('Vikingur Reykjavik', 86, 645), ('Valur Reykjavik', 86, 675)]
[('Iraklis Thessaloniki', 100, 231), ('Aris Thessaloniki', 86, 119), ('PAOK Thessaloniki', 83, 96)]
[('FK Sarajevo', 100, 285), ('Zeljeznicar Sarajevo', 84, 286), ('Slavija Sarajevo', 84, 386), ('Olimpic Sarajevo', 84, 648)]
[('FK Moscow', 100, 351), ('Spartak Moscow', 80, 15), ('Lokomotiv Moscow', 80, 38), ('Dinamo Moscow', 80, 95), ('CSKA Moscow', 80, 135)]
[('KF Tirana', 100, 408), ('Dinam

## looks good to me, these are all explained mostly because larger cities have multiple professional teams. Didn't know there was an Arsendal Kiev. 


In [14]:
# types look good, lots of NaNs
ranks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747 entries, 0 to 746
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   team      747 non-null    object 
 1   coef2000  227 non-null    float64
 2   coef2001  228 non-null    float64
 3   coef2002  230 non-null    float64
 4   coef2003  233 non-null    float64
 5   coef2004  239 non-null    float64
 6   coef2005  237 non-null    float64
 7   coef2006  231 non-null    float64
 8   coef2007  218 non-null    float64
 9   coef2008  209 non-null    float64
 10  coef2009  207 non-null    float64
 11  coef2010  313 non-null    float64
 12  coef2011  365 non-null    float64
 13  coef2012  396 non-null    float64
 14  coef2013  426 non-null    float64
 15  coef2014  453 non-null    float64
 16  coef2015  454 non-null    float64
 17  coef2016  456 non-null    float64
 18  coef2017  453 non-null    float64
 19  coef2018  442 non-null    float64
 20  coef2019  449 non-null    float6

In [15]:
# let's get rid of teams that have more than 15 NAs
ranks['nacounts'] = ranks.isna().sum(axis=1)
ranks = ranks[ranks['nacounts'] < 5]

In [16]:
# nice, got rid of over 600
ranks.shape

(119, 23)

In [17]:
ranks

,team,coef2000,coef2001,coef2002,coef2003,coef2004,coef2005,coef2006,coef2007,coef2008,...,coef2012,coef2013,coef2014,coef2015,coef2016,coef2017,coef2018,coef2019,coef2020,nacounts
0,Juventus,109.963,98.119,91.334,100.155,84.531,93.191,107.020,92.808,66.934,...,46.996,70.829,80.387,95.102,107.087,140.666,126.0,124.0,117.0,0
1,FC Barcelona,103.799,108.605,116.233,140.769,134.350,117.326,127.006,119.374,117.837,...,157.837,157.605,157.542,164.999,159.142,151.999,132.0,138.0,128.0,0
2,Bayern MÃ¼nchen,103.201,110.316,133.495,124.566,105.331,97.166,80.960,73.640,92.078,...,133.037,146.922,154.328,154.883,163.035,154.899,135.0,128.0,136.0,0
3,Real Madrid,99.799,114.605,147.233,151.769,146.350,131.326,120.006,104.374,93.837,...,121.837,136.605,161.542,171.999,176.142,176.999,162.0,146.0,134.0,0
4,Lazio,94.963,105.119,106.334,106.155,83.531,69.191,57.020,51.808,38.934,...,29.996,41.829,52.387,49.102,63.087,56.666,41.0,37.0,41.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Dnipro Dnipropetrovsk,NaN,NaN,13.979,13.291,14.300,24.200,29.777,29.726,31.932,...,14.026,23.951,32.193,52.033,55.476,53.526,34.0,24.0,5.0,2
278,Austria Wien,NaN,NaN,NaN,13.687,8.970,27.208,27.723,30.104,31.840,...,20.765,16.575,23.685,20.635,19.020,17.070,16.0,8.0,10.0,3
287,Villarreal,NaN,NaN,NaN,NaN,42.350,58.326,77.006,78.374,90.837,...,79.837,67.605,53.542,58.999,60.142,64.999,52.0,68.0,56.0,4
289,Manchester City,NaN,NaN,NaN,NaN,26.511,26.864,26.950,28.618,30.996,...,63.882,70.592,72.949,87.078,99.256,100.192,100.0,106.0,116.0,4


# we could probably get rid of more, since most teams haven't had enough games, but we'll remove more later when we combine with matches

In [18]:
# let's save
ranks.to_csv(PATH+'/data/interim/ranks.csv')